In [31]:
import gc
import time
import model_img
import fastText
from keras.callbacks import ModelCheckpoint
from importlib import reload
from keras import backend as K
import sklearn.metrics as sklm

In [4]:
from keras.backend.tensorflow_backend import set_session
import tensorflow as tf

%env CUDA_DEVICE_ORDER=PCI_BUS_ID
%env CUDA_VISIBLE_DEVICES=2

config = tf.ConfigProto()
config.gpu_options.allow_growth = True
set_session(tf.Session(config=config))

env: CUDA_DEVICE_ORDER=PCI_BUS_ID
env: CUDA_VISIBLE_DEVICES=2


In [13]:
reload(model_img)

<module 'model_img' from '/home/gwiedemann/pss-lre2/model_img.py'>

In [14]:
img_dim = (224,224)
model_img.img_path_template = 'data/Tobacco800/images/%s.tif.small.png'

In [8]:
data_train = model_img.read_csv_data("data/Tobacco800/dataset.train", csvformat="Tobacco800")
data_test = model_img.read_csv_data("data/Tobacco800/dataset.test", csvformat="Tobacco800")

In [9]:
model_singlepage = model_img.compile_model_singlepage(img_dim, print_summary=False)

58892288/58889256 [==============================] - 5s 0us/step


In [8]:
# del model_singlepage
# model_singlepage.summary()

In [15]:
model_file = "Tobacco800_exp2_single-page_repeat-%02d.hdf5" % (0,)
checkpoint = model_img.ValidationCheckpoint(model_file, data_test[:200], img_dim, metric='kappa')
model_singlepage.fit_generator(generator=model_img.ImageFeatureGenerator(data_train[:1000], img_dim),
                callbacks = [checkpoint],
                epochs = 5)

Epoch 1/5
32/32 [==============================] - 7s 209ms/step - loss: 3.2040 - acc: 0.5060

kappa improvement: 0.2185730464326162 (before: -inf), saving to Tobacco800_exp2_single-page_repeat-00.hdf5
Epoch 2/5
32/32 [==============================] - 4s 112ms/step - loss: 1.6523 - acc: 0.5959
Epoch 3/5
32/32 [==============================] - 4s 112ms/step - loss: 1.0263 - acc: 0.6573

kappa improvement: 0.385628085573231 (before: 0.2185730464326162), saving to Tobacco800_exp2_single-page_repeat-00.hdf5
Epoch 4/5
32/32 [==============================] - 4s 111ms/step - loss: 0.8507 - acc: 0.7098

kappa improvement: 0.45043103448275856 (before: 0.385628085573231), saving to Tobacco800_exp2_single-page_repeat-00.hdf5
Epoch 5/5
32/32 [==============================] - 4s 113ms/step - loss: 0.7080 - acc: 0.7460


In [16]:
import numpy as np
np.round(model_singlepage.predict_generator(generator=model_img.ImageFeatureGenerator(data_train[:10], img_dim)))

array([[1.],
       [1.],
       [1.],
       [1.],
       [0.],
       [1.],
       [1.],
       [1.],
       [1.],
       [1.]], dtype=float32)

# Experiment: VGG CNN model (single page)
* optimize for Kappa
* 10 repeats

In [17]:
n_repeats = 10
n_epochs = 20
exp_history = []
optimize_for = 'kappa'
for i in range(n_repeats):
    print("Repeat " + str(i+1) + " of " + str(n_repeats))
    print("-------------------------")
    
    if 'model_singlepage' in locals():
        del model_singlepage
        del checkpoint
        gc.collect()
        time.sleep(2)
        K.clear_session()
        time.sleep(2)
    
    model_singlepage = model_img.compile_model_singlepage(img_dim)
    model_file = "Tobacco800_exp2_img_repeat-%02d.hdf5" % (i,)
    checkpoint = model_img.ValidationCheckpoint(model_file, data_test, img_dim, metric=optimize_for)
    model_singlepage.fit_generator(
        generator=model_img.ImageFeatureGenerator(data_train, img_dim),
        callbacks = [checkpoint],
        epochs = n_epochs
    )
    
#     model_slices = model_img.compile_model_singlepage_slices(img_dim, print_summary=False)
#     model_file = "exp1_slices_repeat-%02d.hdf5" % (i,)
#     checkpoint = model_img.ValidationCheckpoint(model_file, data_test, img_dim, prev_page_generator='slices', metric=optimize_for)
#     model_slices.fit_generator(
#         generator=model_img.ImageFeatureGenerator(data_train, img_dim, slices = True),
#         callbacks = [checkpoint],
#         epochs = n_epochs
#     )
    

    with open("current_learning_progress.txt", "a") as pfile:
        pfile.write(str(i) + ": " + str(checkpoint.max_metrics) + '\n')

    exp_history.append(checkpoint.max_metrics)

avg_result = sum([m['kappa'] for m in exp_history]) / n_repeats
avg_acc = sum([m['accuracy'] for m in exp_history]) / n_repeats
print("-------------------------")
print(avg_result)
print(avg_acc)

Repeat 1 of 10
-------------------------
Epoch 1/20
33/33 [==============================] - 5s 149ms/step - loss: 2.9820 - acc: 0.5774

kappa improvement: 0.23939240506329118 (before: -inf), saving to Tobacco800_exp2_img_repeat-00.hdf5
Epoch 2/20
33/33 [==============================] - 4s 115ms/step - loss: 1.6450 - acc: 0.6325

kappa improvement: 0.46012250264567234 (before: 0.23939240506329118), saving to Tobacco800_exp2_img_repeat-00.hdf5
Epoch 3/20
33/33 [==============================] - 4s 111ms/step - loss: 1.1680 - acc: 0.6646

kappa improvement: 0.4620626987732849 (before: 0.46012250264567234), saving to Tobacco800_exp2_img_repeat-00.hdf5
Epoch 4/20
33/33 [==============================] - 4s 115ms/step - loss: 0.8719 - acc: 0.7333

kappa improvement: 0.5443786982248521 (before: 0.4620626987732849), saving to Tobacco800_exp2_img_repeat-00.hdf5
Epoch 5/20
33/33 [==============================] - 4s 114ms/step - loss: 0.6958 - acc: 0.7645

kappa improvement: 0.5686651293876506

Epoch 13/20
33/33 [==============================] - 4s 122ms/step - loss: 0.2807 - acc: 0.8910
Epoch 14/20
33/33 [==============================] - 4s 113ms/step - loss: 0.2268 - acc: 0.9166
Epoch 15/20
33/33 [==============================] - 4s 117ms/step - loss: 0.2161 - acc: 0.9200
Epoch 16/20
33/33 [==============================] - 4s 121ms/step - loss: 0.1948 - acc: 0.9266

kappa improvement: 0.7503963689495445 (before: 0.7342929088817732), saving to Tobacco800_exp2_img_repeat-02.hdf5
Epoch 17/20
33/33 [==============================] - 4s 113ms/step - loss: 0.1638 - acc: 0.9356
Epoch 18/20
33/33 [==============================] - 4s 112ms/step - loss: 0.1653 - acc: 0.9346

kappa improvement: 0.7581398244412625 (before: 0.7503963689495445), saving to Tobacco800_exp2_img_repeat-02.hdf5
Epoch 19/20
33/33 [==============================] - 4s 113ms/step - loss: 0.1663 - acc: 0.9342

kappa improvement: 0.7676842658748569 (before: 0.7581398244412625), saving to Tobacco800_exp2_img_r

Epoch 3/20
33/33 [==============================] - 4s 119ms/step - loss: 0.9688 - acc: 0.6642

kappa improvement: 0.5635376357508426 (before: 0.48665221430783523), saving to Tobacco800_exp2_img_repeat-05.hdf5
Epoch 4/20
33/33 [==============================] - 4s 124ms/step - loss: 0.8223 - acc: 0.6940
Epoch 5/20
33/33 [==============================] - 4s 113ms/step - loss: 0.6560 - acc: 0.7361

kappa improvement: 0.5861971742739889 (before: 0.5635376357508426), saving to Tobacco800_exp2_img_repeat-05.hdf5
Epoch 6/20
33/33 [==============================] - 4s 112ms/step - loss: 0.5369 - acc: 0.7712

kappa improvement: 0.6488568435323843 (before: 0.5861971742739889), saving to Tobacco800_exp2_img_repeat-05.hdf5
Epoch 7/20
33/33 [==============================] - 4s 118ms/step - loss: 0.4900 - acc: 0.7977

kappa improvement: 0.6631155156395169 (before: 0.6488568435323843), saving to Tobacco800_exp2_img_repeat-05.hdf5
Epoch 8/20
33/33 [==============================] - 4s 113ms/step - 

Epoch 11/20
33/33 [==============================] - 4s 115ms/step - loss: 0.3033 - acc: 0.8730

kappa improvement: 0.6310312790337567 (before: 0.6176701529570652), saving to Tobacco800_exp2_img_repeat-07.hdf5
Epoch 12/20
33/33 [==============================] - 4s 123ms/step - loss: 0.2723 - acc: 0.8920

kappa improvement: 0.6528907589216144 (before: 0.6310312790337567), saving to Tobacco800_exp2_img_repeat-07.hdf5
Epoch 13/20
33/33 [==============================] - 4s 114ms/step - loss: 0.3049 - acc: 0.8721

kappa improvement: 0.6715536172713494 (before: 0.6528907589216144), saving to Tobacco800_exp2_img_repeat-07.hdf5
Epoch 14/20
33/33 [==============================] - 4s 113ms/step - loss: 0.2597 - acc: 0.8972

kappa improvement: 0.6936437776255993 (before: 0.6715536172713494), saving to Tobacco800_exp2_img_repeat-07.hdf5
Epoch 15/20
33/33 [==============================] - 4s 113ms/step - loss: 0.2423 - acc: 0.9081
Epoch 16/20
33/33 [==============================] - 4s 112ms/st

Epoch 16/20
33/33 [==============================] - 4s 113ms/step - loss: 0.2130 - acc: 0.9185

kappa improvement: 0.7058354053473309 (before: 0.6967400012325137), saving to Tobacco800_exp2_img_repeat-09.hdf5
Epoch 17/20
33/33 [==============================] - 4s 114ms/step - loss: 0.1747 - acc: 0.9365
Epoch 18/20
33/33 [==============================] - 4s 117ms/step - loss: 0.1711 - acc: 0.9460
Epoch 19/20
33/33 [==============================] - 4s 112ms/step - loss: 0.1586 - acc: 0.9351

kappa improvement: 0.7376369831476195 (before: 0.7058354053473309), saving to Tobacco800_exp2_img_repeat-09.hdf5
Epoch 20/20
33/33 [==============================] - 4s 112ms/step - loss: 0.1476 - acc: 0.9441
-------------------------
0.7201727265030602
0.8648648648648647


In [18]:
for i, r in enumerate(exp_history):
    model_file = "Tobacco800_exp2_img_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + model_file)
print("------------------------- VGG16 non-trainable")
print(avg_result)
print(avg_acc)

0 0.6647513812154696 0.8416988416988417 Tobacco800_exp2_img_repeat-00.hdf5
1 0.6756665954002627 0.8416988416988417 Tobacco800_exp2_img_repeat-01.hdf5
2 0.7837060406421081 0.8957528957528957 Tobacco800_exp2_img_repeat-02.hdf5
3 0.6520013748711059 0.833976833976834 Tobacco800_exp2_img_repeat-03.hdf5
4 0.7013246065139473 0.8571428571428571 Tobacco800_exp2_img_repeat-04.hdf5
5 0.7369749192183412 0.8725868725868726 Tobacco800_exp2_img_repeat-05.hdf5
6 0.7641595337825533 0.8841698841698842 Tobacco800_exp2_img_repeat-06.hdf5
7 0.7433261071539177 0.8764478764478765 Tobacco800_exp2_img_repeat-07.hdf5
8 0.7421797230852765 0.8725868725868726 Tobacco800_exp2_img_repeat-08.hdf5
9 0.7376369831476195 0.8725868725868726 Tobacco800_exp2_img_repeat-09.hdf5
------------------------- VGG16 non-trainable
0.7201727265030602
0.8648648648648647


# Experiment: Predecessor page

*IMPORTANT RESULT*: 
So far, it looks that predecessor image information does not contribute to PSS!

In [21]:
reload(model_img)
model_img.img_path_template = 'data/Tobacco800/images/%s.tif.small.png'

In [22]:
n_repeats = 10
n_epochs = 15
exp2b_history = []
optimize_for = 'kappa'
for i in range(n_repeats):
    print("Repeat " + str(i+1) + " of " + str(n_repeats))
    print("-------------------------")
    
    if 'model_prevpage' in locals():
        del model_prevpage
        del checkpoint
        gc.collect()
        time.sleep(2)
        K.clear_session()
        time.sleep(2)
    
    model_prevpage = model_img.compile_model_prevpage(img_dim)
    model_file = "Tobacco800_exp2_prev-page_repeat-%02d.hdf5" % (i,)
    checkpoint = model_img.ValidationCheckpoint(model_file, data_test, img_dim, prev_page_generator = True, metric=optimize_for)
    model_prevpage.fit_generator(generator=model_img.ImageFeatureGenerator(data_train, img_dim, prevpage=True),
                    callbacks = [checkpoint],
                    epochs = n_epochs)
    exp2b_history.append(checkpoint.max_metrics)
    
    with open("current_learning_progress.txt", "a") as pfile:
        pfile.write(str(i) + ": " + str(checkpoint.max_metrics) + '\n')

avg_kappa = sum([m['kappa'] for m in exp2b_history]) / n_repeats
avg_acc = sum([m['accuracy'] for m in exp2b_history]) / n_repeats
print("-------------------------")
print(avg_kappa)
print(avg_acc)

Repeat 1 of 10
-------------------------
Epoch 1/15
33/33 [==============================] - 8s 254ms/step - loss: 1.6025 - acc: 0.6214

kappa improvement: 0.4478666200006364 (before: -inf), saving to Tobacco800_exp2_prev-page_repeat-00.hdf5
Epoch 2/15
33/33 [==============================] - 7s 207ms/step - loss: 0.5439 - acc: 0.8025

kappa improvement: 0.5293048614266243 (before: 0.4478666200006364), saving to Tobacco800_exp2_prev-page_repeat-00.hdf5
Epoch 3/15
33/33 [==============================] - 7s 210ms/step - loss: 0.2762 - acc: 0.8948

kappa improvement: 0.5874219446922391 (before: 0.5293048614266243), saving to Tobacco800_exp2_prev-page_repeat-00.hdf5
Epoch 4/15
33/33 [==============================] - 7s 223ms/step - loss: 0.2142 - acc: 0.9124

kappa improvement: 0.6281987267507179 (before: 0.5874219446922391), saving to Tobacco800_exp2_prev-page_repeat-00.hdf5
Epoch 5/15
33/33 [==============================] - 7s 215ms/step - loss: 0.1634 - acc: 0.9313

kappa improvement

Epoch 5/15
33/33 [==============================] - 7s 225ms/step - loss: 0.1674 - acc: 0.9422

kappa improvement: 0.6184345375352294 (before: 0.5830650354153251), saving to Tobacco800_exp2_prev-page_repeat-03.hdf5
Epoch 6/15
33/33 [==============================] - 8s 227ms/step - loss: 0.0891 - acc: 0.9687

kappa improvement: 0.6630065691345879 (before: 0.6184345375352294), saving to Tobacco800_exp2_prev-page_repeat-03.hdf5
Epoch 7/15
33/33 [==============================] - 7s 218ms/step - loss: 0.0698 - acc: 0.9754
Epoch 8/15
33/33 [==============================] - 7s 220ms/step - loss: 0.0556 - acc: 0.9810
Epoch 9/15
33/33 [==============================] - 7s 227ms/step - loss: 0.0247 - acc: 0.9924
Epoch 10/15
33/33 [==============================] - 7s 221ms/step - loss: 0.0174 - acc: 0.9953
Epoch 11/15
33/33 [==============================] - 7s 222ms/step - loss: 0.0097 - acc: 0.9972
Epoch 12/15
33/33 [==============================] - 7s 211ms/step - loss: 0.0193 - acc: 0.99

33/33 [==============================] - 7s 221ms/step - loss: 0.0164 - acc: 0.9972
Epoch 13/15
33/33 [==============================] - 7s 212ms/step - loss: 0.0202 - acc: 0.9924

kappa improvement: 0.721034005231574 (before: 0.7189060125895377), saving to Tobacco800_exp2_prev-page_repeat-06.hdf5
Epoch 14/15
33/33 [==============================] - 7s 224ms/step - loss: 0.0123 - acc: 0.9962

kappa improvement: 0.7386807920292597 (before: 0.721034005231574), saving to Tobacco800_exp2_prev-page_repeat-06.hdf5
Epoch 15/15
33/33 [==============================] - 7s 213ms/step - loss: 0.0138 - acc: 0.9962
Repeat 8 of 10
-------------------------
Epoch 1/15
33/33 [==============================] - 9s 261ms/step - loss: 1.5649 - acc: 0.6096

kappa improvement: 0.541505974584308 (before: -inf), saving to Tobacco800_exp2_prev-page_repeat-07.hdf5
Epoch 2/15
33/33 [==============================] - 7s 214ms/step - loss: 0.5650 - acc: 0.7940

kappa improvement: 0.5861971742739889 (before: 0.5415

In [23]:
# -------------------------
# 0.7590367347473503
# 0.8849420849420848
for i, r in enumerate(exp2b_history):
    model_file = "Tobacco800_exp2_prev-page_repeat-%02d.hdf5" % (i,)
    print(str(i) + ' ' + str(r['kappa']) + ' ' + str(r['accuracy']) + ' ' + str(r['f1_macro']) + ' ' + model_file)

0 0.7987067491528586 0.9034749034749034 0.8990882880074808 Tobacco800_exp2_prev-page_repeat-00.hdf5
1 0.775410343759678 0.8918918918918919 0.8875310173697271 Tobacco800_exp2_prev-page_repeat-01.hdf5
2 0.7189060125895377 0.8648648648648649 0.8591888407350452 Tobacco800_exp2_prev-page_repeat-02.hdf5
3 0.6630065691345879 0.8416988416988417 0.8292243611392547 Tobacco800_exp2_prev-page_repeat-03.hdf5
4 0.7393546763947418 0.8764478764478765 0.8686779059449867 Tobacco800_exp2_prev-page_repeat-04.hdf5
5 0.8157495901766108 0.9111969111969112 0.9077592480760595 Tobacco800_exp2_prev-page_repeat-05.hdf5
6 0.7386807920292597 0.8764478764478765 0.8681430571464935 Tobacco800_exp2_prev-page_repeat-06.hdf5
7 0.7987067491528586 0.9034749034749034 0.8990882880074808 Tobacco800_exp2_prev-page_repeat-07.hdf5
8 0.7581398244412625 0.8841698841698842 0.8786998875983514 Tobacco800_exp2_prev-page_repeat-08.hdf5
9 0.7837060406421081 0.8957528957528957 0.8917173781762439 Tobacco800_exp2_prev-page_repeat-09.hdf5


In [35]:
# load best model
model_prevpage.load_weights("Tobacco800_exp2_prev-page_repeat-05.hdf5")

y_predict = np.round(model_prevpage.predict_generator(model_img.ImageFeatureGenerator(data_test, img_dim, prevpage = True)))
y_true = [model_img.label2Idx[x[1]] for x in data_test]
print("Accuracy: " + str(sklm.accuracy_score(y_true, y_predict)))
print("Kappa: " + str(sklm.cohen_kappa_score(y_true, y_predict)))

Accuracy: 0.9111969111969112
Kappa: 0.8157495901766108
